## **Model evaluation for Nigeria flooding study**

In [1]:
import sys; sys.path.append('/home/clair/WWA'); from wwa import *

In [ ]:
# create empty dataframes to hold results & comments
# [eval_df(ens, region) for ens in ["AFR-44", "AFR-22"] for region in ["lake-chad", "lower-niger"]]

---
### **Spatial patterns**

In [2]:
chirps = xr.open_dataset("obs/chirps_spatial_jjas-yearly.nc").precip.mean("time")

sf = pd.concat([gpd.read_file("00_regions/sf_lower-niger"), gpd.read_file("00_regions/sf_lake-chad")])

chirps_cmt = chirps.where(regionmask.mask_3D_geopandas(sf.dissolve(), chirps.longitude, chirps.latitude, drop = False).squeeze(drop = True)).dropna("latitude", "all").dropna("longitude", "all")

In [8]:
ens = "AFR-22"
fl_sp = sorted(glob.glob("cordex/pr-spatial_"+ens+"_*.nc"))

nc = 6
nskip = 1
nr = int(np.ceil((len(fl_sp)+nskip+1) / nc))

plot_kwargs = {"add_colorbar" : True, "cbar_kwargs" : {"location" : "bottom", "fraction" : 0.04, "pad" : 0.05, "label" : ""}, "cmap" : "Blues", "vmin" : 0}

In [9]:
fig, axs = plt.subplots(ncols = nc, nrows = nr, figsize = (nc*3.5, nr*3.5), dpi = 100, subplot_kw = {"projection" : cartopy.crs.PlateCarree()})
for ax in fig.axes: ax.set_axis_off()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load evaluation file
sp_comment_file = ens+"_lower-niger_model-eval.txt"
if len(glob.glob(sp_comment_file)) > 0:
    sp_comments = pd.read_csv(sp_comment_file, quotechar = "'", index_col = "model_name")["spatial_pattern"]
else:
    sp_comments = None
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# CHIRPS data for reference

ax = fig.axes[0]
chirps.plot(ax = ax, **plot_kwargs)
ax.coastlines(lw = 1)
ax.add_feature(cartopy.feature.BORDERS, ls = "--", alpha = 0.5)
ax.set_title("CHIRPS", fontweight = "bold")

# models
for i in range(len(fl_sp)):
    
    fnm = fl_sp[i]
    ax = fig.axes[i+nskip+1]
    mdl = cordex_model(fnm)
    
    pr = convert_units_to(xr.open_dataset(fnm).pr.sel(month = [6,7,8,9]).mean("month"), "mm/day")    
    if cordex_model(fnm) in ["MPI-ESM-MR_r1_RegCM4-3"]: pr = pr * 86400

    if "RegCM" in fnm:
        cbar = ax.pcolormesh(pr.lon, pr.lat, pr, cmap = "Blues", vmin = 0)
        plt.colorbar(cbar, ax = ax, location = "bottom", fraction = 0.04, pad = 0.05)
    else:
        pr.plot(ax = ax, **plot_kwargs)
        
    ax.set_title(cordex_model(fnm))
    ax.coastlines(lw = 1)
    ax.add_feature(cartopy.feature.BORDERS, ls = "--", alpha = 0.5)
    
    # add a coloured block to indicate evaluation
    cmt = "nan"
    if sp_comments is not None:
        if mdl in sp_comments: cmt = re.sub(" .+", "", str(sp_comments[mdl]))
    if cmt == "nan":
        col = "w"
    else:
        col = {"good" : "green", "bad" : "red", "reasonable" : "orange"}[cmt]
    ax.legend(handles = [matplotlib.patches.Patch(color = col, label = "", alpha = 0.5)], loc = "upper left", edgecolor = "w")
    
plt.suptitle("Spatial patterns of JJAS mean precipitation (mm/day) in wider region (CORDEX "+ens+")", fontweight = "bold", y = 0.9)

plt.savefig(ens+"_spatial-patterns_western-Africa.png"); plt.close()

In [10]:
fig, axs = plt.subplots(ncols = nc, nrows = nr, figsize = (nc*3.5, nr*3), dpi = 100, subplot_kw = {"projection" : cartopy.crs.PlateCarree()})
for ax in fig.axes: ax.set_axis_off()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load evaluation file
sp_comment_file = ens+"_lower-niger_model-eval.txt"
if len(glob.glob(sp_comment_file)) > 0:
    sp_comments = pd.read_csv(sp_comment_file, quotechar = "'", index_col = "model_name")["spatial_pattern"]
else:
    sp_comments = None
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# CHIRPS data for reference

ax = fig.axes[0]

chirps_cmt.plot(ax = ax, **plot_kwargs)
ax.coastlines(lw = 1)
ax.add_feature(cartopy.feature.BORDERS, ls = "--", alpha = 0.5)
ax.set_title("CHIRPS", fontweight = "bold")
sf.boundary.plot(ax = ax, color = "k", lw = .5)

# models
for i in range(len(fl_sp)):
    
    fnm = fl_sp[i]
    ax = fig.axes[i+nskip+1]
    mdl = cordex_model(fnm)
    
    pr = convert_units_to(xr.open_dataset(fnm).pr.sel(month = [6,7,8,9]).mean("month"), "mm/day")    
    if cordex_model(fnm) in ["MPI-ESM-MR_r1_RegCM4-3"]: pr = pr * 86400
    
    pr = pr.where(regionmask.mask_3D_geopandas(sf.dissolve(), pr.lon, pr.lat, drop = False).squeeze(drop = True))
    
    if "rlat" in pr.dims:
        pr = pr.dropna("rlat", "all").dropna("rlon", "all")
    elif "x" in pr.dims:
        pr = pr.dropna("x", "all").dropna("y", "all")

    if pr.max() > 20:
        vmax = pr.quantile(.99)
    else:
        vmax = pr.max()
    
    if "RegCM" in fnm:
        cbar = ax.pcolormesh(pr.lon, pr.lat, pr, cmap = "Blues", vmin = 0, vmax = vmax)
        plt.colorbar(cbar, ax = ax, location = "bottom", fraction = 0.04, pad = 0.05)
    else:
        pr.plot(ax = ax, vmax = vmax, **plot_kwargs)
        
    # add a coloured block to indicate evaluation
    cmt = "nan"
    if sp_comments is not None:
        if mdl in sp_comments: cmt = re.sub(" .+", "", str(sp_comments[mdl]))
    if cmt == "nan":
        col = "w"
    else:
        col = {"good" : "green", "bad" : "red", "reasonable" : "orange"}[cmt]    
    ax.legend(handles = [matplotlib.patches.Patch(color = col, label = "", alpha = 0.5)], loc = "upper left", edgecolor = "w")
        
    sf.boundary.plot(ax = ax, color = "k", lw = .5)
    ax.set_title(cordex_model(fnm))
    ax.coastlines(lw = 1)
    ax.add_feature(cartopy.feature.BORDERS, ls = "--", alpha = 0.5)
    
plt.suptitle("Spatial patterns of JJAS mean precipitation (mm/day) in study region (CORDEX "+ens+")", fontweight = "bold", y = 0.9)

plt.savefig(ens+"_spatial-patterns_Niger-Chad.png"); plt.close()

---
### **Seasonal cycle**

In [13]:
ens = "AFR-22"

nc = 5

for rnm in ["chad", "niger", "lagdo"][:1]:
    
    cmt_nm = {"chad" : "Lake Chad", "niger" : "Lower Niger", "lagdo" : "Lagdo Dam"}[rnm]
    comment_file = ens+"_"+re.sub(" ", "-", cmt_nm).lower()+"_model-eval.txt"
    if len(glob.glob(comment_file)) > 0:
        comments = pd.read_csv(comment_file, quotechar = "'", index_col = "model_name")["seasonal_cycle"]
    else:
        comments = None
    
    chirps = xr.open_mfdataset("obs/chirps*"+rnm+"*daily.nc").precip
    obs_dates = slice(str(chirps.time.min().dt.strftime("%Y%m%d").values), str(chirps.time.max().dt.strftime("%Y%m%d").values))
    
    fl = sorted(glob.glob("cordex/pr-"+rnm+"_"+ens+"_*.nc"))
    nr = int(np.ceil(len(fl) / nc))
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    fig, axs = plt.subplots(ncols = nc, nrows = nr, figsize = (nc*3, nr*3), dpi = 100, sharey = True)
    plt.subplots_adjust(hspace = 0.7)
    for ax in fig.axes: ax.set_axis_off()
        
    for i in range(len(fl)):
        
        ax = fig.axes[i]
        fnm = fl[i]
        mdl = cordex_model(fnm)
        
        cmt = "nan"
        if comments is not None:
            if mdl in comments: cmt = re.sub(" .+", "", str(comments[mdl]))
            
        if cmt == "nan":
            col = "tab:blue"
        else:
            col = {"good" : "green", "bad" : "red", "reasonable" : "orange"}[cmt]
        
        pr = xr.open_dataset(fnm).pr.sel(time = obs_dates)
        
        normalised_seasonal_cycle(pr).plot(ax = ax, color = col)
        normalised_seasonal_cycle(chirps).plot(label = "CHIRPS", color = "k", zorder = 100, ax = ax)
        
        [ax.axvline(d, ls = "--", color = "red", alpha = 0.4) for d in [153, 274]]
        
        ax.set_title(mdl)
        ax.set_ylabel("Scaled precip")
        ax.set_axis_on()
        
    plt.suptitle("Scaled seasonal cycle of precipitation in "+ens+" models & CHIRPS ("+cmt_nm+" region)", fontweight = "bold")
    plt.savefig(ens+"_seasonal-cycles-scaled_"+rnm+".png"); plt.close()

## **Model selection for Nigeria flooding study**

#### **Lower Niger catchment: 7-day max precipitation**

In [9]:
# identify models that passed spatial & seasonal evaluation, list filenames

df = pd.concat([pd.read_csv(ens+"_lower-niger_model-eval.txt", quotechar = "'", index_col = "model_name") for ens in ["AFR-44", "AFR-22"]])

df = df.loc[[r[0] & r[1] for r in list(zip([v.split(" ")[0] in ["good", "reasonable"] for v in df.seasonal_cycle.values],
                                           [v.split(" ")[0] in ["good", "reasonable"] for v in df.spatial_pattern.values]))]]
fl = list(set(np.concatenate([glob.glob("cordex/pr-niger_*"+re.sub("_","*",m)+"_*") for m in df.index])))

In [10]:
fl = [fnm for fnm in fl if "MPI" in fnm and "RegCM4-3" in fnm]

In [11]:
for fnm in fl:
    
    csv_fnm = "ts/lower-niger_7daymax_"+cordex_model(fnm)+".txt"

    ts = convert_units_to(xr.open_dataset(fnm).pr, "mm/day")
    
    ts7 = ts.rolling(time = 7, center = False).mean()
    ts7 = ts7.sel(time = [m in [6,7,8,9] for m in ts7.time.dt.month]).resample(time = "AS").max()
    
    # remove any years where exactly equal to zero
    ts7 = ts7.where(ts7 != 0).dropna("time", "any")
        
    csv_str = "# "+re.sub("ts/", "", re.sub(".txt", "", csv_fnm))
    ts7.assign_coords(time = ts7.time.dt.year).rename(time = "#year").to_dataframe().to_csv(csv_fnm, sep = " ")
    
    ! echo "$csv_str" >> $csv_fnm

#### **Lake Chad catchment: JJAS mean precipitation**

In [12]:
# identify models that passed spatial & seasonal evaluation, list filenames

df = pd.concat([pd.read_csv(ens+"_lake-chad_model-eval.txt", quotechar = "'", index_col = "model_name") for ens in ["AFR-44", "AFR-22"]])[["seasonal_cycle"]]
df = df.merge(pd.concat([pd.read_csv(ens+"_lower-niger_model-eval.txt", quotechar = "'", index_col = "model_name") for ens in ["AFR-44", "AFR-22"]])[["spatial_pattern"]],
              left_index = True, right_index = True)

df = df.loc[[r[0] & r[1] for r in list(zip([v.split(" ")[0] in ["good", "reasonable"] for v in df.seasonal_cycle.values],
                                           [v.split(" ")[0] in ["good", "reasonable"] for v in df.spatial_pattern.values]))]]
fl = list(set(np.concatenate([glob.glob("cordex/pr-chad_*"+re.sub("_","*",m)+"_*") for m in df.index])))
fl = [fnm for fnm in fl if "MPI" in fnm and "RegCM4-3" in fnm]

In [13]:
for fnm in fl:
    
    csv_fnm = "ts/lake-chad_jjas-mean_"+cordex_model(fnm)+".txt"
    
    ts = convert_units_to(xr.open_dataset(fnm).pr, "mm/day")
    
    jjas = ts.sel(time = [m in [6,7,8,9] for m in ts.time.dt.month]).resample(time = "AS").mean()

    # remove any years where exactly equal to zero
    jjas = jjas.where(jjas != 0).dropna("time", "any")

    csv_str = "# "+re.sub("ts/", "", re.sub(".txt", "", csv_fnm))
    jjas.assign_coords(time = jjas.time.dt.year).rename(time = "#year").to_dataframe().to_csv(csv_fnm, sep = " ")
    
    ! echo "$csv_str" >> $csv_fnm


In [8]:
# which GSATs do I need to upload?
set([fnm.split("_")[2]+"_"+fnm.split("_")[3] for fnm in glob.glob("ts/*")])

{'EC-EARTH_r12',
 'HadGEM2-ES_r1',
 'IPSL-CM5A-LR_r1',
 'MIROC5_r1',
 'MPI-ESM-LR_r1',
 'MPI-ESM-MR_r1',
 'NorESM1-M_r1'}